In [4]:
import pathlib
import warnings

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import (
    StratifiedKFold,
    KFold,
    GridSearchCV,
)
from sklearn.utils import shuffle, parallel_backend
from sklearn.exceptions import ConvergenceWarning
from joblib import dump

import sys

sys.path.append("../utils")
from split_utils import get_features_data
from train_utils import get_dataset, get_X_y_data

In [5]:
# set numpy seed to make random operations reproduceable
np.random.seed(0)

# load training data from indexes and features dataframe
data_split_path = pathlib.Path(f"../1.split_data/indexes/data_split_indexes.tsv")
features_dataframe_path = pathlib.Path("../0.download_data/data/labeled_data.csv.gz")

# dataframe with only the labeled data we want (exclude certain phenotypic classes)
features_dataframe = get_features_data(features_dataframe_path)
data_split_indexes = pd.read_csv(data_split_path, sep="\t", index_col=0)

# get training data from labeled data
training_data = get_dataset(features_dataframe, data_split_indexes, "train")
#training_data

In [6]:
# specify model types and feature types
model_types = ["final", "shuffled_baseline"]
feature_types = ["CP", "DP", "CP_and_DP"]

# create stratified data sets for k-fold cross validation
#straified_k_folds = StratifiedKFold(n_splits=10, shuffle=False)
k_folds = KFold(n_splits=10, shuffle=False)

# create logistic regression model with following parameters
log_reg_model = LogisticRegression(
    penalty="elasticnet",
    solver="saga",
    max_iter=10,
    n_jobs=-1,
    random_state=0,  # CHANGE MAX ITER TO 100
)

# specify parameters to tune for
parameters = {"C": np.logspace(-3, 3, 7), "l1_ratio": np.linspace(0, 1, 11)}
print(f"Parameters being tested during grid search: {parameters}\n")

# create grid search with cross validation with hypertuning params
grid_search_cv = GridSearchCV(
    log_reg_model, parameters, cv=k_folds, n_jobs=-1, scoring="f1_weighted"
)

# train model on each combination of model type and feature type
for model_type in model_types:
    for feature_type in feature_types:
        for phenotypic_class in training_data["Mitocheck_Phenotypic_Class"].unique():
            # create results directory
            results_dir = pathlib.Path(
                f"models/single_class_models/{phenotypic_class}_models/"
            )
            results_dir.mkdir(parents=True, exist_ok=True)

            # get number of labels for this specific phenotypic_class
            phenotypic_class_counts = (
                training_data.loc[
                    training_data["Mitocheck_Phenotypic_Class"] == phenotypic_class
                ]
            ).shape[0]
            print(
                f"Training {model_type} model on {feature_type} features for {phenotypic_class} phenotypic class with {phenotypic_class_counts} positive labels..."
            )

            # create deep copy of training data so we can make modifications without affecting original training data
            class_training_data = training_data.copy(deep=True)
            # convert labels that are not phenotypic class to 0 (negative)
            class_training_data.loc[
                class_training_data["Mitocheck_Phenotypic_Class"] != phenotypic_class,
                "Mitocheck_Phenotypic_Class",
            ] = f"Not {phenotypic_class}"
            # convert labels that are phenotypic class to 1 (positive)
            # class_training_data.loc[
            #     class_training_data["Mitocheck_Phenotypic_Class"] == phenotypic_class,
            #     "Mitocheck_Phenotypic_Class",
            # ] = 1

            # get X (features) and y (labels) data
            X, y = get_X_y_data(class_training_data, feature_type)
            print(f"X has shape {X.shape}, y has shape {y.shape}")
            
            lb = preprocessing.LabelBinarizer()
            y = lb.fit_transform(y).ravel()
            
            

            # shuffle columns of X (features) dataframe independently to create shuffled baseline
            if model_type == "shuffled_baseline":
                for column in X.T:
                    np.random.shuffle(column)
                print("Shuffled!")

            # fit grid search cv to X and y data
            # capture convergence warning from sklearn
            # this warning does not affect the model but takes up lots of space in the output
            with parallel_backend("multiprocessing"):
                with warnings.catch_warnings():
                    warnings.filterwarnings(
                        "ignore", category=ConvergenceWarning, module="sklearn"
                    )
                    grid_search_cv = grid_search_cv.fit(X, y)

            # print info for best estimator
            print(f"Best parameters: {grid_search_cv.best_params_}")
            print(f"Score of best estimator: {grid_search_cv.best_score_}\n")

            # save final estimator
            # dump(
            #     grid_search_cv.best_estimator_,
            #     f"{results_dir}/{model_type}__{feature_type}.joblib",
            # )
            break
        break
    # break
    


Parameters being tested during grid search: {'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])}

Training final model on CP features for Large phenotypic class with 67 positive labels...
X has shape (2432, 165), y has shape (2432,)
Best parameters: {'C': 0.1, 'l1_ratio': 0.0}
Score of best estimator: 0.9609192844525735

Training shuffled_baseline model on CP features for Large phenotypic class with 67 positive labels...
X has shape (2432, 165), y has shape (2432,)
Shuffled!
Best parameters: {'C': 0.001, 'l1_ratio': 0.0}
Score of best estimator: 0.9589204291297457

